In [2]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['AWS_PROFILE'] = 'usman_server'
server_host = os.getenv('server_host')
print(server_host)

ec2-54-153-96-137.us-west-1.compute.amazonaws.com


In [4]:
import mlflow

In [5]:
mlflow.set_tracking_uri(f"http://{server_host}:5000")

In [6]:
print(f"tracking URI: {mlflow.get_tracking_uri()}")

tracking URI: http://ec2-54-153-96-137.us-west-1.compute.amazonaws.com:5000


In [7]:
mlflow.set_experiment('usman-server-ride-experiment')


<Experiment: artifact_location='s3://mlflow-artifact-zoomcamp/1', experiment_id='1', lifecycle_stage='active', name='usman-server-ride-experiment', tags={}>

In [8]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [9]:
df_train = read_dataframe('../../week_2/Data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('../../week_2/Data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)


In [ ]:
def main(y_train, y_val, dict_train, dict_val):
    with mlflow.start_run():
        params = dict(max_depth= 20, n_estimators=100, min_samples_leaf=10, random_state = 0)
        mlflow.log_params(params)

        pipeline = make_pipeline(DictVectorizer(), RandomForestRegressor(**params, n_jobs= -1))

        pipeline.fit(dict_train, y_train)

        y_pred = pipeline.predict(dict_val)

        rmse = mean_squared_error(y_pred, y_val, squared=False)
        print(params, rmse)

        mlflow.log_metric('rmse', rmse)

        mlflow.sklearn.log_model(pipeline, artifact_path = 'model')


In [10]:
with mlflow.start_run():
    params = dict(max_depth= 20, n_estimators=100, min_samples_leaf=10, random_state = 0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs= -1)

    X_train = dv.fit_transform(dict_train)
    model.fit(X_train, y_train)

    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    print(params, rmse)

    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(model, artifact_path = 'model')

    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


c:\Users\Usman\.virtualenvs\web_service_mlflow-JzKsfKyc\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [11]:
from mlflow.tracking import MlflowClient

tracking_uri = mlflow.get_tracking_uri()
run_id= 'c2f6c8d09b61440e890f65149b5777fe'

# Load model as a PyFuncModel.
client = MlflowClient(tracking_uri = tracking_uri)

path = client.download_artifacts(run_id=run_id, path='dict_vectorizer.bin')
print(f'Downloading the dict vectorizer to {path}')

with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)
print(dv)

DictVectorizer()


In [12]:
path = client.download_artifacts(run_id=run_id, path='dict_vectorizer.bin')


In [14]:
from sklearn.pipeline import make_pipeline


In [25]:
def prepare_features(ride):
    features = {}
    features['PU_DO'] = '%s_%s' % (ride['PULocationID'], ride['DOLocationID'])
    features['trip_distance'] = ride['trip_distance']
    return features
ride = {
    "PULocationID": 10,
    "DOLocationID": 50,
    "trip_distance": 40
}
features= prepare_features(ride)
features
with mlflow.start_run():
    params = dict(max_depth= 20, n_estimators=100, min_samples_leaf=10, random_state = 0)
    mlflow.log_params(params)

    pipeline = make_pipeline(DictVectorizer(), RandomForestRegressor(**params, n_jobs= -1))

    pipeline.fit(dict_train, y_train)

    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)

    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path = 'model')



{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


In [26]:
run_id= '540cc07b5ef34c45aab9fb61837f12e9'
logged_model = f"s3://mlflow-artifact-zoomcamp/1/{run_id}/artifacts/model"
model = mlflow.pyfunc.load_model(logged_model)
model.predict(features)[0]

45.50965007660852